In [32]:
import pandas as pd

In [33]:
accounts = pd.read_csv("../Dataset/accounts.csv")
products = pd.read_csv("../Dataset/products.csv")
sales_teams = pd.read_csv("../Dataset/sales_teams.csv")
pipeline = pd.read_csv("../Dataset/sales_pipeline.csv")

In [34]:
#Quality Check 

def audit(data, name):
    print(f"\n{name.upper()}")
    print(data.info())
    print(data.isnull().sum())

audit(accounts, "accounts")



ACCOUNTS
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   account           85 non-null     object 
 1   sector            85 non-null     object 
 2   year_established  85 non-null     int64  
 3   revenue           85 non-null     float64
 4   employees         85 non-null     int64  
 5   office_location   85 non-null     object 
 6   subsidiary_of     15 non-null     object 
dtypes: float64(1), int64(2), object(4)
memory usage: 4.8+ KB
None
account              0
sector               0
year_established     0
revenue              0
employees            0
office_location      0
subsidiary_of       70
dtype: int64


In [35]:
audit(products, "products")



PRODUCTS
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   product      7 non-null      object
 1   series       7 non-null      object
 2   sales_price  7 non-null      int64 
dtypes: int64(1), object(2)
memory usage: 300.0+ bytes
None
product        0
series         0
sales_price    0
dtype: int64


In [36]:
audit(sales_teams, "sales_teams")



SALES_TEAMS
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   sales_agent      35 non-null     object
 1   manager          35 non-null     object
 2   regional_office  35 non-null     object
dtypes: object(3)
memory usage: 972.0+ bytes
None
sales_agent        0
manager            0
regional_office    0
dtype: int64


In [37]:
audit(pipeline, "sales_pipeline")


SALES_PIPELINE
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8800 entries, 0 to 8799
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   opportunity_id  8800 non-null   object 
 1   sales_agent     8800 non-null   object 
 2   product         8800 non-null   object 
 3   account         7375 non-null   object 
 4   deal_stage      8800 non-null   object 
 5   engage_date     8300 non-null   object 
 6   close_date      6711 non-null   object 
 7   close_value     6711 non-null   float64
dtypes: float64(1), object(7)
memory usage: 550.1+ KB
None
opportunity_id       0
sales_agent          0
product              0
account           1425
deal_stage           0
engage_date        500
close_date        2089
close_value       2089
dtype: int64


In [38]:
#Converting numeric values in accounts table 

accounts['revenue'] = pd.to_numeric(accounts['revenue'], errors='coerce')
accounts['employees'] = pd.to_numeric(accounts['employees'], errors='coerce')
accounts['year_established'] = pd.to_numeric(accounts['year_established'], errors='coerce')

accounts['sector'] = accounts['sector'].fillna('Unknown')


In [39]:
#product table converting ints to numeric value 

products['sales_price'] = pd.to_numeric(products['sales_price'], errors='coerce')

products.dropna(subset=['product'], inplace=True)


In [40]:
#filling null values 

sales_teams[['sales_agent','manager','regional_office']] = \
sales_teams[['sales_agent','manager','regional_office']].fillna('Unknown')


In [41]:
#sales pipleline converting date to date values 

pipeline['engage_date'] = pd.to_datetime(pipeline['engage_date'], errors='coerce')
pipeline['close_date'] = pd.to_datetime(pipeline['close_date'], errors='coerce')

pipeline['close_value'] = pd.to_numeric(pipeline['close_value'], errors='coerce').fillna(0)
pipeline['deal_stage'] = pipeline['deal_stage'].str.strip().str.title()


In [42]:
pipeline['is_won'] = pipeline['close_value'] > 0


In [43]:
pipeline['sales_cycle_days'] = (
    pipeline['close_date'] - pipeline['engage_date']
).dt.days

In [44]:

pipeline['deal_outcome'] = pipeline['is_won'].map({True: 'Won', False: 'Lost'})


In [45]:
data = (
    pipeline
    .merge(accounts, on='account', how='left')
    .merge(products, on='product', how='left')
    .merge(sales_teams, on='sales_agent', how='left')
)


In [46]:
data

,opportunity_id,sales_agent,product,account,deal_stage,engage_date,close_date,close_value,is_won,sales_cycle_days,...,sector,year_established,revenue,employees,office_location,subsidiary_of,series,sales_price,manager,regional_office
0,1C1I7A6R,Moses Frase,GTX Plus Basic,Cancity,Won,2016-10-20,2017-03-01,1054.0,True,132.0,...,retail,2001.0,718.62,2448.0,United States,NaN,GTX,1096.0,Dustin Brinkmann,Central
1,Z063OYW0,Darcel Schlecht,GTXPro,Isdom,Won,2016-10-25,2017-03-11,4514.0,True,137.0,...,medical,2002.0,3178.24,4540.0,United States,NaN,NaN,NaN,Melvin Marxen,Central
2,EC4QE1BX,Darcel Schlecht,MG Special,Cancity,Won,2016-10-25,2017-03-07,50.0,True,133.0,...,retail,2001.0,718.62,2448.0,United States,NaN,MG,55.0,Melvin Marxen,Central
3,MV1LWRNH,Moses Frase,GTX Basic,Codehow,Won,2016-10-25,2017-03-09,588.0,True,135.0,...,software,1998.0,2714.90,2641.0,United States,Acme Corporation,GTX,550.0,Dustin Brinkmann,Central
4,PE84CX4O,Zane Levy,GTX Basic,Hatfan,Won,2016-10-25,2017-03-02,517.0,True,128.0,...,services,1982.0,792.46,1299.0,United States,NaN,GTX,550.0,Summer Sewald,West
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8795,9MIWFW5J,Versie Hillebrand,MG Advanced,NaN,Prospecting,NaT,NaT,0.0,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,MG,3393.0,Dustin Brinkmann,Central
8796,6SLKZ8FI,Versie Hillebrand,MG Advanced,NaN,Prospecting,NaT,NaT,0.0,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,MG,3393.0,Dustin Brinkmann,Central
8797,LIB4KUZJ,Versie Hillebrand,MG Advanced,NaN,Prospecting,NaT,NaT,0.0,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,MG,3393.0,Dustin Brinkmann,Central
8798,18IUIUK0,Versie Hillebrand,MG Advanced,NaN,Prospecting,NaT,NaT,0.0,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,MG,3393.0,Dustin Brinkmann,Central


In [47]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8800 entries, 0 to 8799
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   opportunity_id    8800 non-null   object        
 1   sales_agent       8800 non-null   object        
 2   product           8800 non-null   object        
 3   account           7375 non-null   object        
 4   deal_stage        8800 non-null   object        
 5   engage_date       8300 non-null   datetime64[ns]
 6   close_date        6711 non-null   datetime64[ns]
 7   close_value       8800 non-null   float64       
 8   is_won            8800 non-null   bool          
 9   sales_cycle_days  6711 non-null   float64       
 10  deal_outcome      8800 non-null   object        
 11  sector            7375 non-null   object        
 12  year_established  7375 non-null   float64       
 13  revenue           7375 non-null   float64       
 14  employees         7375 n

In [48]:
data.isnull().sum().sort_values(ascending=False)

subsidiary_of       7508
close_date          2089
sales_cycle_days    2089
sales_price         1480
series              1480
employees           1425
account             1425
office_location     1425
sector              1425
year_established    1425
revenue             1425
engage_date          500
opportunity_id         0
manager                0
deal_outcome           0
sales_agent            0
is_won                 0
close_value            0
deal_stage             0
product                0
regional_office        0
dtype: int64

In [49]:
data['deal_status'] = data['close_date'].apply(
    lambda x: 'Closed' if pd.notnull(x) else 'Open'
)

data['deal_status'].value_counts()


deal_status
Closed    6711
Open      2089
Name: count, dtype: int64

In [50]:
data.head()

,opportunity_id,sales_agent,product,account,deal_stage,engage_date,close_date,close_value,is_won,sales_cycle_days,...,year_established,revenue,employees,office_location,subsidiary_of,series,sales_price,manager,regional_office,deal_status
0,1C1I7A6R,Moses Frase,GTX Plus Basic,Cancity,Won,2016-10-20,2017-03-01,1054.0,True,132.0,...,2001.0,718.62,2448.0,United States,NaN,GTX,1096.0,Dustin Brinkmann,Central,Closed
1,Z063OYW0,Darcel Schlecht,GTXPro,Isdom,Won,2016-10-25,2017-03-11,4514.0,True,137.0,...,2002.0,3178.24,4540.0,United States,NaN,NaN,NaN,Melvin Marxen,Central,Closed
2,EC4QE1BX,Darcel Schlecht,MG Special,Cancity,Won,2016-10-25,2017-03-07,50.0,True,133.0,...,2001.0,718.62,2448.0,United States,NaN,MG,55.0,Melvin Marxen,Central,Closed
3,MV1LWRNH,Moses Frase,GTX Basic,Codehow,Won,2016-10-25,2017-03-09,588.0,True,135.0,...,1998.0,2714.90,2641.0,United States,Acme Corporation,GTX,550.0,Dustin Brinkmann,Central,Closed
4,PE84CX4O,Zane Levy,GTX Basic,Hatfan,Won,2016-10-25,2017-03-02,517.0,True,128.0,...,1982.0,792.46,1299.0,United States,NaN,GTX,550.0,Summer Sewald,West,Closed


In [51]:
revenue_data = data.copy()

In [52]:
revenue_data.head()

,opportunity_id,sales_agent,product,account,deal_stage,engage_date,close_date,close_value,is_won,sales_cycle_days,...,year_established,revenue,employees,office_location,subsidiary_of,series,sales_price,manager,regional_office,deal_status
0,1C1I7A6R,Moses Frase,GTX Plus Basic,Cancity,Won,2016-10-20,2017-03-01,1054.0,True,132.0,...,2001.0,718.62,2448.0,United States,NaN,GTX,1096.0,Dustin Brinkmann,Central,Closed
1,Z063OYW0,Darcel Schlecht,GTXPro,Isdom,Won,2016-10-25,2017-03-11,4514.0,True,137.0,...,2002.0,3178.24,4540.0,United States,NaN,NaN,NaN,Melvin Marxen,Central,Closed
2,EC4QE1BX,Darcel Schlecht,MG Special,Cancity,Won,2016-10-25,2017-03-07,50.0,True,133.0,...,2001.0,718.62,2448.0,United States,NaN,MG,55.0,Melvin Marxen,Central,Closed
3,MV1LWRNH,Moses Frase,GTX Basic,Codehow,Won,2016-10-25,2017-03-09,588.0,True,135.0,...,1998.0,2714.90,2641.0,United States,Acme Corporation,GTX,550.0,Dustin Brinkmann,Central,Closed
4,PE84CX4O,Zane Levy,GTX Basic,Hatfan,Won,2016-10-25,2017-03-02,517.0,True,128.0,...,1982.0,792.46,1299.0,United States,NaN,GTX,550.0,Summer Sewald,West,Closed


In [53]:
revenue_data.to_csv("crm_clea_master_data", index=False)